In [1]:
require 'daru'
require 'daru_plotly'
include Daru::Plotly::Initializer
require 'statsample'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

In [2]:
# Read the data set into a daru DataFrame
wine = Daru::DataFrame.from_csv('./winequality-both.csv')
wine.vectors = Daru::Index.new(wine.vectors.map { |name| name.gsub(' ', '_') })
wine.head

#<Daru::DataFrame(10x13)>
                  type fixed_acid volatile_a citric_aci residual_s  chlorides free_sulfu total_sulf    density         pH  sulphates    alcohol    quality
          0        red        7.4        0.7          0        1.9      0.076         11         34     0.9978       3.51       0.56        9.4          5
          1        red        7.8       0.88          0        2.6      0.098         25         67     0.9968        3.2       0.68        9.8          5
          2        red        7.8       0.76       0.04        2.3      0.092         15         54      0.997       3.26       0.65        9.8          5
          3        red       11.2       0.28       0.56        1.9      0.075         17         60      0.998       3.16       0.58        9.8          6
          4        red        7.4        0.7          0        1.9      0.076         11         34     0.9978       3.51       0.56        9.4          5
          5        red        7.4       0.66          0        1.8      0.075         13         40     0.9978       3.51       0.56        9.4          5
          6        red        7.9        0.6       0.06        1.6      0.069         15         59     0.9964        3.3       0.46        9.4          5
          7        red        7.3       0.65          0        1.2      0.065         15         21     0.9946       3.39       0.47         10          7
          8        red        7.8       0.58       0.02          2      0.073          9         18     0.9968       3.36       0.57        9.5          7
          9        red        7.5        0.5       0.36        6.1      0.071         17        102     0.9978       3.35        0.8       10.5          5

In [3]:
# Display descriptive statistics for all variables
wine.describe

#<Daru::DataFrame(5x12)>
               alcohol  chlorides citric_aci    density fixed_acid free_sulfu         pH    quality residual_s  sulphates total_sulf volatile_a
      count       6497       6497       6497       6497       6497       6497       6497       6497       6497       6497       6497       6497
       mean 10.4918008 0.05603386 0.31863321 0.99469663 7.21530706 30.5253193 3.21850084 5.81837771 5.44323533 0.53126827 115.744574 0.33966599
        std 1.19271174 0.03503360 0.14531786 0.00299867 1.29643375 17.7493997 0.16078720 0.87325527 4.75780374 0.14880587 56.5218545 0.16463647
        min          8      0.009          0    0.98711        3.8          1       2.72          3        0.6       0.22          6       0.08
        max       14.9      0.611       1.66    1.03898       15.9        289       4.01          9       65.8          2        440       1.58

In [4]:
# Identify unique values
wine.quality.uniq.sort

#<Daru::Vector(7)>
         quality
     459       3
      18       4
       0       5
       3       6
       7       7
     267       8
    2373       9

In [5]:
# Calculate value frequencies
wine.quality.value_counts

#<Daru::Vector(7)>
    5 2138
    6 2836
    7 1079
    4  216
    8  193
    3   30
    9    5

In [6]:
# Display descriptive statistics for quality by wine type
# wine.group_by('type')[['alcohol']].descrive.unstack('type')

In [7]:
# Calculate specific quantiles
# wine.groupby('type')[['quality']].quantile([0.25, 0.75]).unstack('type')

In [8]:
wine.corr

#<Daru::DataFrame(12x12)>
            fixed_acid volatile_a citric_aci residual_s  chlorides free_sulfu total_sulf    density         pH  sulphates    alcohol    quality
 fixed_acid        1.0 0.21900825 0.32443572 -0.1119812 0.29819477 -0.2827354 -0.3290539 0.45890998 -0.2527004 0.29956774 -0.0954515 -0.0767432
 volatile_a 0.21900825 1.00000000 -0.3779813 -0.1960111 0.37712427 -0.3525573 -0.4144761 0.27129564 0.26145440 0.22598367 -0.0376403 -0.2656994
 citric_aci 0.32443572 -0.3779813        1.0 0.14245122 0.03899801 0.13312580 0.19524197 0.09615392 -0.3298081 0.05619730 -0.0104934 0.08553171
 residual_s -0.1119812 -0.1960111 0.14245122 0.99999999 -0.1289404 0.40287064 0.49548158 0.55251695 -0.2673198 -0.1859274 -0.3594147 -0.0369804
  chlorides 0.29819477 0.37712427 0.03899801 -0.1289404 1.00000000 -0.1950447 -0.2796304 0.36261465 0.04470797 0.39559330 -0.2569155 -0.2006655
 free_sulfu -0.2827354 -0.3525573 0.13312580 0.40287064 -0.1950447 1.00000000 0.72093408 0.02571684 -0.1458538 -0.1884572 -0.1798384 0.05546305
 total_sulf -0.3290539 -0.4144761 0.19524197 0.49548158 -0.2796304 0.72093408        1.0 0.03239451 -0.2384131 -0.2757268 -0.2657396 -0.0413854
    density 0.45890998 0.27129564 0.09615392 0.55251695 0.36261465 0.02571684 0.03239451        1.0 0.01168608 0.25947849 -0.6867454 -0.3058579
         pH -0.2527004 0.26145440 -0.3298081 -0.2673198 0.04470797 -0.1458538 -0.2384131 0.01168608 1.00000000 0.19212340 0.12124846 0.01950570
  sulphates 0.29956774 0.22598367 0.05619730 -0.1859274 0.39559330 -0.1884572 -0.2757268 0.25947849 0.19212340 0.99999999 -0.0030291 0.03848544
    alcohol -0.0954515 -0.0376403 -0.0104934 -0.3594147 -0.2569155 -0.1798384 -0.2657396 -0.6867454 0.12124846 -0.0030291        1.0 0.44431852
    quality -0.0767432 -0.2656994 0.08553171 -0.0369804 -0.2006655 0.05546305 -0.0413854 -0.3058579 0.01950570 0.03848544 0.44431852        1.0

In [9]:
# Look at relationship between pairs of variables
# Take a "small" sample of red and white wines for plotting
def take_sample(data_frame, replace: false, n: 200)
  index_array = data_frame.index.to_a
  if replace
    index_to_fetch = (1..n).map { index_array.sample }
  else
    index_to_fetch = index_array.sample(n)
  end
  data_frame.row[*index_to_fetch]
end

reds = wine.where(wine['type'].eq 'red')
whites = wine.where(wine['type'].eq 'white')
reds_sample = take_sample(reds)
whites_sample = take_sample(whites)
data_source = reds_sample.vectors.map do |vect|
  [vect, reds_sample[vect].to_a + whites_sample[vect].to_a]
end.to_h
wine_sample = Daru::DataFrame.new(data_source)
wine.add_vector 'in_sample', Daru::Vector.new(wine.index.map { |i| reds_sample.index.include?(i) || whites_sample.index.include?(i) ? 1.0 : 0.0 })

wine['in_sample'].head

#<Daru::Vector(10)>
   0 0.0
   1 0.0
   2 0.0
   3 0.0
   4 0.0
   5 0.0
   6 0.0
   7 0.0
   8 0.0
   9 0.0

In [10]:
wine.pivot_table(index: ['in_sample', 'type'], agg: :count)
#print(pd.crosstab(wine.in_sample, wine.type, margins=True))

#<Daru::DataFrame(4x12)>
                       fixed_acid volatile_a citric_aci residual_s  chlorides free_sulfu total_sulf    density         pH  sulphates    alcohol    quality
        0.0        red       1399       1399       1399       1399       1399       1399       1399       1399       1399       1399       1399       1399
                 white       4698       4698       4698       4698       4698       4698       4698       4698       4698       4698       4698       4698
        1.0        red        200        200        200        200        200        200        200        200        200        200        200        200
                 white        200        200        200        200        200        200        200        200        200        200        200        200

In [11]:
#sns.set_style("dark")
#sns.set_style("darkgrid", {"legend.scatterpoints": 0})
#pg = sns.PairGrid(wine_sample, hue="type", hue_order=["red", "white"], \
#palette=dict(red="red", white="white"), hue_kws={"marker": ["o", "s"]}, vars=['quality', 'alcohol', 'residual_sugar'])
#pg.x = wine_sample.ix[wine_sample['type']=='red', 'quality']
#pg = pg.map_diag(plt.hist)
#pg.x = wine_sample.ix[wine_sample['type']=='white', 'quality']
#pg = pg.map_diag(plt.hist)
#pg = pg.map_offdiag(plt.scatter, edgecolor="black", s=10, alpha=0.25)

In [12]:
#g = sns.pairplot(wine_sample, kind='reg', plot_kws={"ci": False, "x_jitter": 0.25, "y_jitter": 0.25}, \
#hue='type', diag_kind='hist', diag_kws={"bins": 10, "alpha": 1.0}, palette=dict(red="red", white="white"), \
#markers=["o", "s"], vars=['quality', 'alcohol', 'residual_sugar'])
#sns.set_style({'legend.frameon': True,'legend.numpoints': 0,'legend.scatterpoints': 0})
#wine_all_plot = sns.pairplot(wine, kind='reg', hue='type', palette=dict(red="red", white="white"), markers=["o", "s"], vars=['quality', 'alcohol', 'residual_sugar'])
#wine_sample_plot = sns.pairplot(wine_sample, kind='reg', hue='type', palette=dict(red="red", white="white"), markers=["o", "s"], vars=['quality', 'alcohol', 'residual_sugar'])

#wine['ln_fixed_acidity'] = np.log(wine.ix[:, 'fixed_acidity'])
#sns.distplot(wine.ix[:, 'fixed_acidity'])
#sns.distplot(wine.ix[:, 'ln_fixed_acidity'])
#print(g)
#plt.suptitle('Histograms and Scatter Plots of Quality, Alcohol, and Residual Sugar', fontsize=14, \
#		horizontalalignment='center', verticalalignment='top',
#		x=0.5, y=0.999)

In [13]:
# Look at the distribution of quality by wine type
red_wine = wine.where(wine['type'].eq 'red')['quality']
white_wine = wine.where(wine['type'].eq 'white')['quality']

trace_red = {
  x: red_wine.to_a,
  type: :histogram,
  histnorm: :density,
  opacity: 0.5,
  marker: {
    color: 'red',
  }
}

trace_white = {
  x: white_wine.to_a,
  type: :histogram,
  histnorm: :density,
  opacity: 0.5,
  marker: {
    color: 'gray',
  }
}

data = [trace_red, trace_white]
Plotly::Plot.new(data: data).show

#<Plotly::Offline::HTML:0x007fd8b37eae30 @id="a8b4d224-4d0f-41ff-a0ae-587ef6c0a9c3", @data=[{:x=>[5, 5, 5, 6, 5, 5, 5, 7, 7, 5, 5, 5, 5, 5, 5, 5, 7, 5, 4, 6, 6, 5, 5, 5, 6, 5, 5, 5, 5, 6, 5, 6, 5, 6, 5, 6, 6, 7, 4, 5, 5, 4, 6, 5, 5, 4, 5, 5, 5, 5, 5, 6, 6, 5, 6, 5, 5, 5, 5, 6, 5, 5, 7, 5, 5, 5, 5, 5, 5, 6, 6, 5, 5, 4, 5, 5, 5, 6, 5, 4, 5, 5, 5, 5, 6, 5, 6, 5, 5, 5, 5, 6, 5, 5, 4, 6, 5, 5, 5, 6, 6, 6, 6, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 6, 5, 6, 6, 6, 6, 6, 5, 6, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 6, 6, 5, 5, 5, 5, 5, 5, 5, 6, 5, 6, 5, 5, 5, 6, 6, 6, 4, 5, 5, 5, 5, 5, 5, 5, 6, 5, 4, 6, 5, 5, 5, 5, 4, 6, 5, 4, 6, 6, 6, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 6, 7, 4, 7, 5, 5, 5, 6, 7, 7, 5, 5, 7, 6, 6, 6, 5, 6, 5, 5, 5, 5, 5, 6, 5, 5, 6, 4, 6, 6, 5, 6, 5, 7, 6, 6, 5, 6, 6, 6, 6, 6, 6, 5, 6, 6, 7, 7, 6, 5, 5, 6, 6, 6, 6, 5, 5, 6, 5, 5, 5, 5, 7, 5, 4, 5, 5, 5, 7, 4, 8, 6, 6, 6, 6, 5, 5, 5, 6, 6, 6, 8, 7, 6, 7, 5, 7, 5, 5, 6, 6, 7, 5, 7, 5, 6, 6, 6, 5, 5, 5, 5, 5, 6, 6, 5, 5, 5, 6, 5, 6, 6, 6, 6, 6, 6, 5, 5, 6, 5, 6, 7, 6, 7, 5, 5, 6, 6, 6, 7, 5, 6, 5, 6, 6, 6, 5, 7, 7, 6, 5, 6, 7, 6, 6, 6, 6, 6, 5, 7, 6, 6, 6, 6, 6, 5, 5, 6, 6, 5, 7, 7, 6, 5, 6, 5, 5, 7, 6, 7, 5, 5, 7, 5, 6, 6, 5, 6, 7, 6, 7, 6, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 7, 8, 6, 5, 5, 5, 7, 5, 6, 6, 5, 5, 6, 6, 6, 5, 6, 6, 7, 6, 4, 6, 5, 5, 7, 5, 5, 6, 5, 6, 5, 7, 7, 5, 7, 5, 7, 6, 6, 5, 6, 7, 5, 6, 5, 6, 5, 6, 6, 6, 5, 8, 6, 7, 7, 7, 6, 5, 5, 6, 6, 6, 6, 6, 7, 5, 8, 5, 5, 7, 3, 6, 5, 5, 5, 6, 5, 6, 6, 6, 5, 5, 6, 6, 5, 6, 5, 5, 6, 5, 6, 5, 8, 5, 5, 6, 5, 5, 6, 7, 6, 6, 7, 7, 6, 6, 8, 6, 5, 8, 6, 6, 7, 7, 7, 7, 7, 7, 6, 6, 7, 5, 6, 6, 7, 7, 5, 6, 3, 6, 5, 6, 5, 5, 5, 5, 5, 5, 6, 6, 5, 6, 5, 5, 6, 6, 6, 5, 6, 7, 5, 5, 6, 5, 6, 6, 5, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 6, 5, 6, 6, 5, 5, 5, 6, 6, 5, 6, 6, 6, 6, 6, 6, 5, 4, 6, 6, 4, 5, 5, 6, 5, 5, 5, 7, 7, 6, 7, 5, 8, 7, 5, 6, 5, 5, 5, 5, 6, 6, 6, 6, 4, 6, 5, 6, 6, 6, 7, 6, 6, 6, 5, 5, 6, 5, 6, 5, 5, 6, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 6, 5, 6, 5, 6, 4, 5, 5, 5, 5, 7, 6, 5, 5, 5, 5, 5, 7, 5, 4, 7, 6, 5, 5, 5, 6, 5, 5, 5, 7, 6, 4, 6, 5, 6, 6, 5, 5, 6, 6, 5, 6, 5, 5, 5, 5, 6, 5, 6, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 6, 6, 6, 5, 6, 6, 6, 6, 4, 4, 5, 5, 5, 6, 6, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 6, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 6, 5, 5, 5, 5, 6, 6, 5, 6, 6, 5, 5, 5, 5, 6, 6, 6, 5, 5, 5, 5, 5, 6, 5, 6, 6, 5, 5, 6, 5, 6, 5, 5, 6, 6, 5, 6, 6, 5, 5, 6, 5, 5, 5, 5, 5, 5, 6, 6, 5, 6, 5, 6, 5, 6, 5, 5, 7, 6, 6, 5, 5, 7, 6, 6, 7, 7, 7, 5, 6, 5, 6, 5, 4, 6, 5, 6, 6, 5, 5, 5, 7, 5, 5, 5, 5, 7, 5, 8, 6, 4, 6, 3, 4, 5, 5, 7, 7, 7, 5, 7, 5, 6, 5, 6, 5, 5, 6, 5, 5, 5, 5, 5, 6, 6, 7, 6, 7, 7, 6, 5, 6, 5, 5, 5, 5, 6, 6, 6, 6, 6, 5, 4, 7, 7, 7, 4, 6, 6, 5, 5, 6, 6, 5, 6, 5, 6, 7, 6, 5, 5, 5, 6, 5, 6, 6, 7, 6, 7, 3, 5, 7, 7, 7, 7, 5, 5, 6, 6, 6, 6, 6, 6, 7, 6, 6, 5, 6, 6, 6, 5, 6, 6, 6, 5, 7, 6, 4, 5, 7, 5, 5, 6, 5, 5, 6, 6, 4, 7, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 5, 6, 6, 7, 5, 6, 5, 5, 6, 6, 6, 7, 5, 6, 5, 6, 6, 7, 5, 7, 5, 5, 5, 7, 5, 6, 5, 6, 6, 5, 6, 7, 5, 5, 6, 5, 5, 6, 5, 5, 6, 7, 7, 6, 6, 7, 7, 7, 7, 5, 7, 7, 7, 7, 5, 7, 6, 5, 6, 6, 6, 7, 6, 6, 5, 6, 6, 5, 6, 7, 6, 6, 5, 6, 7, 7, 7, 5, 6, 6, 7, 7, 5, 7, 6, 5, 6, 6, 7, 6, 6, 6, 5, 6, 6, 5, 5, 5, 7, 6, 6, 7, 5, 7, 7, 6, 8, 6, 6, 6, 6, 7, 7, 7, 5, 7, 5, 6, 6, 5, 7, 6, 5, 5, 7, 6, 7, 6, 6, 6, 5, 7, 6, 7, 7, 8, 6, 6, 7, 6, 5, 6, 5, 7, 5, 6, 6, 6, 6, 6, 5, 6, 7, 5, 6, 6, 7, 6, 6, 6, 6, 6, 6, 6, 5, 8, 6, 6, 6, 4, 7, 6, 6, 5, 6, 6, 5, 7, 7, 7, 6, 6, 6, 5, 6, 6, 6, 6, 6, 5, 6, 6, 7, 6, 6, 7, 6, 5, 6, 6, 5, 7, 7, 6, 5, 7, 6, 7, 5, 5, 5, 5, 7, 6, 6, 6, 6, 6, 6, 6, 6, 4, 7, 5, 6, 6, 5, 6, 5, 5, 6, 5, 6, 5, 4, 6, 5, 7, 5, 6, 6, 6, 6, 6, 6, 6, 7, 8, 5, 7, 7, 7, 5, 7, 7, 6, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 7, 5, 6, 5, 5, 4, 6, 4, 6, 6, 4, 4, 5, 5, 6, 5, 6, 5, 5, 5, 6, 6, 6, 5, 5, 5, 5, 5, 5, 6, 6, 6, 5, 4, 5, 4, 6, 6, 6, 6, 6, 8, 6, 6, 5, 5, 6, 6, 4, 6, 6, 7, 6, 6, 6, 6, 5, 5, 6, 5, 5, 5, 5, 6, 6, 4, 6, 5, 5, 6, 6, 3, 6,

In [14]:
puts 'std'
wine.group_by('type').std['quality']

std


#<Daru::Vector(2)>
                               quality
                red 0.8075694397347023
              white 0.8856385749678454

In [15]:
puts 'mean'
wine.group_by('type').mean['quality']

mean


#<Daru::Vector(2)>
                               quality
                red 5.6360225140712945
              white   5.87790935075541

In [18]:
Statsample::Analysis.store(Statsample::Test::T) do
  
  
  a=rnorm(10)
  t_1=Statsample::Test.t_one_sample(a,{:u=>50})
  summary t_1
  
  b=rnorm(10,2)
  
  t_2=Statsample::Test.t_two_samples_independent(a,b)
  summary t_2
end
Statsample::Analysis.run_batch

NOTE: Daru::Vector#n_valid is deprecated; use count_values instead. It will be removed on or after 2016-10-01.
Daru::Vector#n_valid called from /Users/sangenya/.rbenv/versions/2.3.2/lib/ruby/gems/2.3.0/gems/statsample-2.0.2/lib/statsample/test/t.rb:166.
NOTE: Daru::Vector#n_valid is deprecated; use count_values instead. It will be removed on or after 2016-10-01.
Daru::Vector#n_valid called from /Users/sangenya/.rbenv/versions/2.3.2/lib/ruby/gems/2.3.0/gems/statsample-2.0.2/lib/statsample/test/t.rb:170.
NOTE: Daru::Vector#n_valid is deprecated; use count_values instead. It will be removed on or after 2016-10-01.
Daru::Vector#n_valid called from /Users/sangenya/.rbenv/versions/2.3.2/lib/ruby/gems/2.3.0/gems/statsample-2.0.2/lib/statsample/test/t.rb:170.
NOTE: Daru::Vector#n_valid is deprecated; use count_values instead. It will be removed on or after 2016-10-01.
Daru::Vector#n_valid called from /Users/sangenya/.rbenv/versions/2.3.2/lib/ruby/gems/2.3.0/gems/statsample-2.0.2/lib/statsample

Analysis 2017-03-09 00:54:37 +0900
= Statsample::Test::T
  == One Sample T Test
    Sample mean: -0.1333 | Sample sd: 0.8610 | se : 0.2723
    Population mean: 50.0000
    t(9) = -184.1383, p=-0.0000 (both tails)
    CI(95%): -50.7492 - -49.5174
  == Two Sample T Test
    Mean and standard deviation
+----------+---------+--------+----+
| Variable |  mean   |   sd   | n  |
+----------+---------+--------+----+
|          | -0.1333 | 0.8610 | 10 |
|          | 2.1058  | 0.7457 | 10 |
+----------+---------+--------+----+

    Levene test for equality of variances : F(1, 18) = 0.0878 , p = 0.7704
    T statistics
+--------------------+---------+---------+----------------+
|        Type        |    t    |   df    | p (both tails) |
+--------------------+---------+---------+----------------+
| Equal variance     | -6.2165 | 18      | 0.0000         |
| Non equal variance | -6.2165 | 17.6406 | 0.0000         |
+--------------------+---------+---------+----------------+

    Effect size
+------